<a href="https://colab.research.google.com/github/LamaAlQarni/bert4rec_on_taobao/blob/main/1_TaoBao_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!gdown 1STmDw9gtVJlImN36-yGM7g_53xhmMOAM -O UserBehavior.csv

Downloading...
From (original): https://drive.google.com/uc?id=1STmDw9gtVJlImN36-yGM7g_53xhmMOAM
From (redirected): https://drive.google.com/uc?id=1STmDw9gtVJlImN36-yGM7g_53xhmMOAM&confirm=t&uuid=195887b7-2acd-4bdf-bf93-fd19e534eddf
To: /content/UserBehavior.csv
100% 3.67G/3.67G [00:59<00:00, 61.3MB/s]


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
column_names = ['user_id', 'item_id', 'category_id', 'behavior_type', 'timestamp']
taobao_df=pd.read_csv('UserBehavior.csv',nrows=80000000,names=column_names)
taobao_df

,user_id,item_id,category_id,behavior_type,timestamp
0,1,2268318,2520377,pv,1511544070
1,1,2333346,2520771,pv,1511561733
2,1,2576651,149192,pv,1511572885
3,1,3830808,4181361,pv,1511593493
4,1,4365585,2520377,pv,1511596146
...,...,...,...,...,...
79999995,992781,2194211,2355072,pv,1512197874
79999996,992781,1160864,1859277,pv,1512198229
79999997,992781,4053215,1859277,pv,1512198242
79999998,992781,1389888,1859277,pv,1512198682


In [5]:
taobao_df=taobao_df[taobao_df['behavior_type']=='buy']
taobao_df

,user_id,item_id,category_id,behavior_type,timestamp
71,100,1603476,2951233,buy,1511579908
73,100,2971043,4869428,buy,1511617549
100,100,598929,2429887,buy,1511759458
119,100,1046201,3002561,buy,1511767298
125,100,1606258,4098232,buy,1511789940
...,...,...,...,...,...
79999679,992756,2164689,4244487,buy,1511770679
79999696,992756,3337988,3776866,buy,1512006282
79999705,992756,2290158,1451814,buy,1512114154
79999721,992768,1357749,1884322,buy,1511684003


In [6]:
user_sequences=taobao_df.groupby('user_id').apply(
    lambda x: x.sort_values('timestamp')['item_id'].tolist()).to_dict()
print(f"Number of Users Interactions = {len(user_sequences)}")

<ipython-input-6-f11d252a210b>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  user_sequences=taobao_df.groupby('user_id').apply(


Number of Users Interactions = 536925


In [7]:
seq_lens=[len(user_seq) for user_seq in user_sequences.values()]

In [8]:
print(f"Min Sequence Length = {np.min(seq_lens)}")
print(f"Max Sequence Length = {np.max(seq_lens)}")
print(f"Mean Sequence Length = {np.mean(seq_lens)}")

Min Sequence Length = 1
Max Sequence Length = 177
Mean Sequence Length = 3.0005233505610653


In [9]:
users_ids=[]
required_len=10
for user_id,user_seq in user_sequences.items():
  if len(user_seq)>=required_len:
    users_ids.append(user_id)

print(f"Number of Users Interactions (>= {required_len} interaction) = {len(users_ids)}")

Number of Users Interactions (>= 10 interaction) = 16469


In [10]:
taobao_df=taobao_df.set_index('user_id').iloc[users_ids]
taobao_df.reset_index(inplace=True)

In [11]:
taobao_df

,user_id,item_id,category_id,behavior_type,timestamp
0,1000085,280390,2633883,buy,1511771239
1,1000085,327684,794153,buy,1512131522
2,1000296,2596189,1297992,buy,1512100060
3,100030,341797,965809,buy,1511626467
4,1000488,3339260,982926,buy,1511780214
...,...,...,...,...,...
16464,569892,3548330,982926,buy,1511902530
16465,570249,3561544,982926,buy,1512263675
16466,570321,2216091,4015820,buy,1511758526
16467,570386,2097163,3736316,buy,1512007662


In [12]:
taobao_df.to_csv('taobao_df.csv',index=False)